In [1]:
%load_ext autoreload
%autoreload 2
from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt

failed to setup nemotron api
[Errno 2] No such file or directory: 'env/nemotron.key'


In [2]:
import yaml

In [3]:
conf_path="config.yaml"

with open(conf_path,"r") as f:
    conf=yaml.safe_load(f.read())
client_dict=get_client_dict(conf)
#print(launch_command(conf))

client_dict

{'weblab-GENIAC/Tanuki-8B-dpo-v1.0': {'client': <openai.OpenAI at 0x7f71fc162f90>,
  'config': {'name': 'weblab-GENIAC/Tanuki-8B-dpo-v1.0',
   'PORT': 8000,
   'max-model-len': 2048,
   'gpu-memory-utilization': 0.3,
   'GPU_ID': 0,
   'template1': '<s>以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n',
   'template2': '\n\n### 応答:\n',
   'max_tokens': 1000}},
 'cyberagent/calm3-22b-chat': {'client': <openai.OpenAI at 0x7f71fc1944d0>,
  'config': {'name': 'cyberagent/calm3-22b-chat',
   'PORT': 8001,
   'max-model-len': 3000,
   'gpu-memory-utilization': 0.8,
   'GPU_ID': 1,
   'template1': '<|im_start|>system\nあなたは親切なAIアシスタントです。<|im_end|>\n<|im_start|>user\n',
   'template2': '<|im_end|>\n<|im_start|>assistant',
   'max_tokens': 2000}},
 'team-hatakeyama-phase2/Tanuki-8x8B-dpo-v1.0': {'client': <openai.OpenAI at 0x7f71fc190250>,
  'config': {'name': 'team-hatakeyama-phase2/Tanuki-8x8B-dpo-v1.0',
   'PORT': 8002,
   'max-model-len': 2048,
   'gpu-memory-utilization': 0.8,
   'GPU_ID': '

In [5]:
from src.Client import Client
with open("env/url.txt") as f:
    url = f.read().strip()


# apiクライアントとchatbotを起動
client = Client(url)

In [6]:
client.get_q_and_a()

(['質問',
  '狐と狸は､どちらがカントを読むのに適しているか',
  'googleColabとAWS SageMakerのメリット・デメリットを教えて。',
  '架空の会社の貸借対照表を作成し、計算するためのpythonコードを示してくれますか？',
  '日本語の文法で「複合語」とは何か',
  '新卒会計社員教育用の架空会社の貸借対照表を作成してくれますか？\n\n１．資産\n受取手形、売掛金、貸付金、有価証券、商品\n２．負債\n買掛金、支払手形、借入金\n３．純資産\nその会社が毎年稼いだ利益剰余金や設立時に投資した資本金\n\n- テーブル構造で作成してください\n- 年間売り上げ10億程度の会社を想定しています。\n- 製造業',
  '「株式会社テクノクラフト」の貸借対照表\n\n| 項目 | 額（万円） | 備考 |\n|----|----|----|\n| 受取手形 | 3,000,000 | 売上の30%を未収 |\n| 売掛金 | 45,000,000 | 未収商品やサービス |\n| 貸付金 | 100,000 | 関連会社に対する貸付金 |\n| 有価証券 | 20,000,000 | 投資有価証券など |\n| 商品 | 160,000,000 | 製造途中の品や在庫 |\n| 合計 | 270,000,000 | |\n\n| 項目 | 額（万円） | 備考 |\n|----|----|----|\n| 買掛金 | 150,000,000 | 製造に必要な材料の購入費用 |\n| 支払手形 | 200,000 | 仕入れ先に対する未払金 |\n| 借入金 | 850,000 | 運転資金や設備投資の借入金 |\n| 合計 | 197,200,000 | |\n\n| 項目 | 額（万円） | 備考 |\n|----|----|----|\n| 資本金 | 400,000 | 設立時の投資 |\n| 利益剰余金 | 610,400 | 毎年稼いだ利益 |\n| 合計 | 1,010,400 | |\n\n| 合計 | 366,200,000 | |\n\n上記は架空会社の貸借対照表です。\n\nこの会社の財務状況を批評してください。\nまた、この会社が赤字となる場合、どのような数値になると考えられますか？',
  '架空の会社

In [38]:
from collections import Counter
import random
win_models=[i[8] for i in client.values if len(i[8])>4]
lose_models=[i[9] for i in client.values if len(i[9])>4]
model_history=win_models+lose_models

model_counter=Counter(model_history)

model_counter

#もっとも少ないモデルを選択
target_model_name=model_counter.most_common()[-1][0]

#選択確率を1/頻度にして､確率的に選択
target_model_name=random.choices([i for i in model_counter.keys()],weights=[1/i[1] for i in model_counter.items()])[0]

other_models=[i for i in client_dict.keys() if i!=target_model_name]

enemy_model_name=random.choice(other_models)

if random.random()<0.5:
    target_model_name,enemy_model_name=enemy_model_name,target_model_name

target_model_name,enemy_model_name

('tokyotech-llm/Llama-3-Swallow-70B-Instruct-v0.1', 'gpt-4o-2024-05-13')

In [40]:
import google.generativeai as genai
try:
    with open("env/google.key", "r") as f:
        GOOGLE_API_KEY= f.read().strip()
    genai.configure(api_key=GOOGLE_API_KEY)
    gemini_pro = genai.GenerativeModel("gemini-1.5-pro")
except Exception as e:
    print("failed to setup google api")
    print(e)



In [42]:
prompt = "純粋理性批判はたぬきに理解できますか"
response = gemini_pro.generate_content(prompt)
print(response.text)

 たぬきがカントの純粋理性批判を理解できるかどうかは、難しい問題です。カントの哲学は非常に難解であり、人間にとっても理解するのは容易ではありません。たぬきは人間よりも知能が低いと考えられているため、カントの哲学を理解することはさらに難しいでしょう。

しかし、たぬきは人間よりも自然に近い存在であるため、カントの哲学の根底にある思想、つまり人間の理性には限界があるということを、直感的に理解できるかもしれません。例えば、たぬきは人間のように複雑な言語や数学を理解することはできませんが、それでも自然の中で生き抜くための知恵や能力を持っています。これは、人間が理性によって理解できる世界には限界があり、人間の理性を超えた世界が存在することを示唆しているのかもしれません。

結論として、たぬきがカントの純粋理性批判を完全に理解できるかどうかはわかりません。しかし、たぬきは人間とは異なる視点から世界を捉えているため、カントの哲学に新しい光を当てることができるかもしれません。


In [4]:
model_name="hatakeyama-llm-team/Tanuki-8B-Instruct"
model_name="microsoft/Phi-3-medium-128k-instruct"
#model_name="Rakuten/RakutenAI-7B-chat"
#model_name="karakuri-ai/karakuri-lm-8x7b-chat-v0.1"
#model_name="Qwen/Qwen2-7B-Instruct"
#model_name="models--mistralai--Mixtral-8x22B-Instruct-v0.1"
model_name="llm-jp/llm-jp-13b-instruct-full-ac_001_16x-dolly-ichikara_004_001_single-oasst-oasst2-v2.0"
model_name="nvidia/Nemotron-4-340B-Instruct"

question="純粋理性批判はたぬきに理解できますか?"
#question="以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。### 指示:\n"
res=ask_llm_prompt(client_dict,model_name,question)
res

NameError: name 'nemotron' is not defined

In [ ]:
prompt="以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。### 指示:\n元気ですか\n\n### 応答:\n"
client = client_dict[model_name]["client"]
completion = client.completions.create(model=model_name,
                                        prompt=prompt,
                                        max_tokens=int(client_dict[model_name]["config"]["max_tokens"]))
out=completion.choices[0].text.strip()  # .message.content.strip()
out


In [ ]:
cmd="""以下のテキストをもとに､日本語で教科書風の文章を生成しなさい｡
-[問題文]を出力
-[考え方]を出力
-[答え]を出力
-[詳細な解説]を出力
"""
#cmd="以下のテキストをもとに､複数の登場人物でなされる日本語の会話文を生成しなさい｡"
question=cmd+"""
#テキスト
Q.
Rick took off on a road trip for the summer. He traveled to the first destination on his trip, and then from there, he traveled twice the distance to this second destination. The third point on his trip was 40 miles away, which was half the distance he traveled from the beginning to his first destination. The final destination took twice the amount of driving of all the other legs put together. How many miles in total did he travel across all the legs of the trip?

A.	
Let's solve this problem using Python's sympy library. <llm-code> import sympy as sp # third leg was 40 miles x = 40 # third leg was half the first leg first_leg = 2 * x # second leg was twice the first leg second_leg = 2 * first_leg # fourth leg was equal to the sum of the three legs fourth_leg = x + first_leg + second_leg # adding up the four to get the total distance total_distance = x + first_leg + second_leg + fourth_leg total_distance </llm-code> <llm-code-output> 560 </llm-code-output> Thus Rick traveled \boxed{560} miles in total.

"""
res=ask_llm_prompt(client_dict,model_name,question)
print(res)

In [ ]:
model_name

In [ ]:
client = client_dict[model_name]["client"]

template1 = client_dict[model_name]["config"]["template1"]
template2 = client_dict[model_name]["config"]["template2"]
prompt = template1+question+template2
completion = client.completions.create(model=model_name,
                                        prompt=prompt,
                                        max_tokens=int(client_dict[model_name]["config"]["max_tokens"]))


In [ ]:

from datasets import load_dataset
ds=load_dataset("tokyotech-llm/Swallow-Instruct-v0.1",split="train")
ds

In [ ]:
q_list=[]

for record in ds:
    for conv in record["conversation"]:
        if conv["role"]=="user":
            q_list.append(conv["content"])

In [ ]:
import pandas as pd
df=pd.DataFrame(q_list)
df.to_csv("swallow.csv")